In [2]:
import requests
import json
import pandas as pd

In [4]:
def get_emotion(questiontext):
    mymodel = 'Emotion_en'
    myverbose = 'y'
    outputformat = 'json'
    debugsetting ='n'
    print(questiontext)
    response = requests.post('https://api.meaningcloud.com/deepcategorization-1.0', 
                            data=[('key', 'ed81d00b312901e5ec1dcc308375effa'),
                                  ('txt', questiontext),
                                  ('of', outputformat),
                                  ('model', mymodel),
                                  ('verbose', myverbose),
                                  ('debug', debugsetting)])
    if response.status_code != 200:
        print("POST failed with status code " + response.status)

    return response


In [88]:
#Test 
result = get_emotion('I was really upset because I crashed my car.  I was really happy because my mother called me.  My brother was also really upset because his dog died.')
print(result)
result.content
      

I was really upset because I crashed my car.  I was really happy because my mother called me.  My brother was also really upset because his dog died.
<Response [200]>


b'{"status":{"code":"0","msg":"OK","credits":"1","remaining_credits":"18457"},"category_list":[{"code":"Sadness","label":"Sadness","abs_relevance":"5","relevance":"100","term_list":[{"form":"was upset","abs_relevance":"1","offset_list":[{"inip":"2","endp":"17"}]},{"form":"crashed","abs_relevance":"1","offset_list":[{"inip":"29","endp":"35"}]},{"form":"upset","abs_relevance":"1","offset_list":[{"inip":"122","endp":"126"}]},{"form":"really was upset","abs_relevance":"1","offset_list":[{"inip":"6","endp":"17"}]},{"form":"really upset","abs_relevance":"1","offset_list":[{"inip":"115","endp":"126"}]}]},{"code":"Joy","label":"Joy","abs_relevance":"2","relevance":"40","term_list":[{"form":"happy","abs_relevance":"1","offset_list":[{"inip":"59","endp":"63"}]},{"form":"really happy","abs_relevance":"1","offset_list":[{"inip":"52","endp":"63"}]}]}]}'

In [56]:
result.content

b'{"status":{"code":"0","msg":"OK","credits":"1","remaining_credits":"19960"},"category_list":[{"code":"Sadness","label":"Sadness","abs_relevance":"5","relevance":"100","term_list":[{"form":"was upset","abs_relevance":"1","offset_list":[{"inip":"2","endp":"17"}]},{"form":"crashed","abs_relevance":"1","offset_list":[{"inip":"29","endp":"35"}]},{"form":"upset","abs_relevance":"1","offset_list":[{"inip":"122","endp":"126"}]},{"form":"really was upset","abs_relevance":"1","offset_list":[{"inip":"6","endp":"17"}]},{"form":"really upset","abs_relevance":"1","offset_list":[{"inip":"115","endp":"126"}]}]},{"code":"Joy","label":"Joy","abs_relevance":"2","relevance":"40","term_list":[{"form":"happy","abs_relevance":"1","offset_list":[{"inip":"59","endp":"63"}]},{"form":"really happy","abs_relevance":"1","offset_list":[{"inip":"52","endp":"63"}]}]}]}'

In [10]:
result_dict = json.loads(result.content)
category_list = result_dict['category_list']
category_list
meaning_cloud_response_df = pd.DataFrame(category_list)
meaning_cloud_response_df

,code,label,abs_relevance,relevance,term_list
0,Sadness,Sadness,5,100,"[{'form': 'was upset', 'abs_relevance': '1', '..."
1,Joy,Joy,2,40,"[{'form': 'happy', 'abs_relevance': '1', 'offs..."


Label	Description
Anger	Texts that express annoyance, displeasure, hostility.
Anticipation	Texts that express expectation, prediction.
Disgust	Texts that express revulsion, disapproval.
Fear	Texts that express threat, danger, pain, harm.
Joy	Texts that express pleasure, happiness.
Sadness	Texts that express sorrow, unhappiness.
Surprise	Texts that express unexpectedness, astonishing.
Trust	Texts that express reliability, truth, ability.

In [77]:
categories = pd.json_normalize(data=result_dict['category_list'])
categories

,code,label,abs_relevance,relevance,term_list
0,Sadness,Sadness,5,100,"[{'form': 'was upset', 'abs_relevance': '1', '..."
1,Joy,Joy,2,40,"[{'form': 'happy', 'abs_relevance': '1', 'offs..."


In [52]:
for c_index, c_row in categories.iterrows():
        term_list_data = pd.json_normalize(c_row.loc['term_list'])
        print("Code: " , c_row.loc['code'], term_list_data) 


Code:  Sadness                form abs_relevance                       offset_list
0         was upset             1     [{'inip': '2', 'endp': '17'}]
1           crashed             1    [{'inip': '29', 'endp': '35'}]
2             upset             1  [{'inip': '122', 'endp': '126'}]
3  really was upset             1     [{'inip': '6', 'endp': '17'}]
4      really upset             1  [{'inip': '115', 'endp': '126'}]
Code:  Joy            form abs_relevance                     offset_list
0         happy             1  [{'inip': '59', 'endp': '63'}]
1  really happy             1  [{'inip': '52', 'endp': '63'}]


In [79]:
result_pd = pd.DataFrame()
extract_pd = pd.DataFrame()
question = 1
questionText = "this is my question text"

for c_index, c_row in categories.iterrows():
    category = c_row.loc['label']
    c_term_list = pd.json_normalize(c_row.loc['term_list'])
    for index, row in c_term_list.iterrows():
        print(row)
        extract_pd = pd.json_normalize(row.loc['offset_list'])
        extract_pd['VisionAPIquestionID'] = question
        extract_pd['QuestionTExt'] = questionText
        extract_pd['form'] = row.loc ['form']
        extract_pd['abs_relevance'] = row.loc['abs_relevance']
        extract_pd['EmotionLabel'] = category
        result_pd = result_pd.append(extract_pd)

result_pd

form                                 was upset
abs_relevance                                1
offset_list      [{'inip': '2', 'endp': '17'}]
Name: 0, dtype: object
form                                    crashed
abs_relevance                                 1
offset_list      [{'inip': '29', 'endp': '35'}]
Name: 1, dtype: object
form                                        upset
abs_relevance                                   1
offset_list      [{'inip': '122', 'endp': '126'}]
Name: 2, dtype: object
form                          really was upset
abs_relevance                                1
offset_list      [{'inip': '6', 'endp': '17'}]
Name: 3, dtype: object
form                                 really upset
abs_relevance                                   1
offset_list      [{'inip': '115', 'endp': '126'}]
Name: 4, dtype: object
form                                      happy
abs_relevance                                 1
offset_list      [{'inip': '59', 'endp': '63'}]
Name: 0, dtype:

,inip,endp,form,abs_relevance,EmotionLabel
0,2,17,was upset,1,Sadness
0,29,35,crashed,1,Sadness
0,122,126,upset,1,Sadness
0,6,17,really was upset,1,Sadness
0,115,126,really upset,1,Sadness
0,59,63,happy,1,Joy
0,52,63,really happy,1,Joy


In [34]:
question_pd = pd.json_normalize(result_dict['category_list'])
question_pd
# code
# label
# abs_relevance
# relevance



,code,label,abs_relevance,relevance,term_list
0,Anger,Anger,2,100,"[{'form': 'angry', 'abs_relevance': '1', 'offs..."


In [57]:
#Nicely formated timespan from Jeff Heaton
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60*60))
    m = int((sec_elapsed % (60*60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [58]:
#Read in the visionAPIClassificationFile - the important columns are "ID" and "Question Text"
import pandas as pd
visionapi_df = pd.DataFrame
filename = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\VisionAPIQuestions_source.csv'
visionapi_df = pd.read_csv(filename, sep=',', )

In [59]:
testtext = visionapi_df.loc[1, 'Question Text']
print(testtext)

"I have problem with access to camera first time. I installed my app. Next go to scan to QR Code. I use to scan QR Code google vision. App show dialog, which show about the permissions, next click "Allow",but camera doesn't open. But I go back activity and go to activity which scan QR Code, camera open.my AdnroidManifest.xmlmy class"


In [60]:
visionapi_df.head()

,ID,FearID,fear,JoyID,joy,LoveID,love,SadID,Sadness,SurpriseID,surprise,AngerID,anger,NoEmotionID,none,NoOfEmotions,Include,Question Text
0,45033467,45033467,fear,45033467,NoJoy,45033467,love,45033467,notsadness,45033467,notsurprise,45033467,NotAnger,45033467,NaN,2.0,Yes,"""I want to build a cloud based solution in whi..."
1,45546462,45546462,fear,45546462,NoJoy,45546462,notlove,45546462,notsadness,45546462,notsurprise,45546462,NotAnger,45546462,NaN,1.0,YES,"""I have problem with access to camera first ti..."
2,49619897,49619897,notfear,49619897,NoJoy,49619897,notlove,49619897,notsadness,49619897,surprise,49619897,Anger,49619897,NaN,2.0,Yes,"""Using Airflow I want to get the result of an ..."
3,55388663,55388663,notfear,55388663,NoJoy,55388663,notlove,55388663,notsadness,55388663,notsurprise,55388663,NotAnger,55388663,NoEmotion,0.0,Yes,"""I try to use Google Cloud Vision API to detec..."
4,52833231,52833231,notfear,52833231,NoJoy,52833231,notlove,52833231,notsadness,52833231,notsurprise,52833231,NotAnger,52833231,NoEmotion,0.0,Yes,"""How can i write a test case in Junit for amaz..."


In [87]:
#Process the Vision API dataframe and collect the data into a list [Question no, meaningcloud response.content]
import time
totalCount = 0
start_time = time.time()
categoryfn = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\MeaningCloud_categories.csv'
termsfn = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\MeaningCloud_terms.csv'

iteration_df = visionapi_df.copy()  # take a copy of the dataframe

category_df = pd.DataFrame(columns = ['QuestionID', 'Question Text',
                                      'TrustCategory', 'TrustAbsRelevance', 'TrustRelevance',
                                      'JoyCategory', 'JoyAbsRelevance', 'JoyRelevance',
                                      'AnticipationCategory', 'AnticipationAbsRelevance', 'AnticipationRelevance',
                                      'SurpriseCategory', 'SurpriseAbsRelevance', 'SurpriseRelevance',
                                      'SadnessCategory', 'SadnessAbsRelevance', 'SadnessRelevance',
                                      'FearCategory', 'FearAbsRelevance', 'FearRelevance',
                                      'AngerCategory', 'AngerAbsRelevance', 'AngerRelevance',
                                      'DisgustCategory', 'DisgustAbsRelevance', 'DisgustRelevance',
                                      'NoEmotion'])
                                      
all_results = []
category_emotion_pd = pd.DataFrame()

termlist_df = pd.DataFrame(columns = ['QuestionID', 'Question Text', 'Emotion Label', 'Form',
                                      'AbsoluteRelevance','inip','endp'])

termlist_emotion_df = pd.DataFrame()

for index, row in iteration_df.iterrows():
    totalCount+=1
    if index >=0 :
        ta = get_emotion(row.loc['Question Text']) 
        #all_results[index] = ta
        #ta = result  #meaningcloud result
        ta_result_dict = json.loads(ta.content)  #json to dictionary
        result1 = ta_result_dict
        #result = meaningcloud response 
        #print("result ", result1)
        #print("question", row.loc['Question Text'])
    
        TrustCategory = False
        TrustAbsRelevance = 0
        TrustRelevance = 0  
        JoyCategory = False
        JoyAbsRelevance = 0
        JoyRelevance = 0
        AnticipationCategory = False
        AnticipationAbsRelevance = 0
        AnticipationRelevance = 0
        SurpriseCategory = False
        SurpriseAbsRelevance = 0
        SurpriseRelevance = 0
        SadnessCategory = False
        SadnessAbsRelevance = 0
        SadnessRelevance = 0
        FearCategory = False
        FearAbsRelevance = 0
        FearRelevance = 0                           
        AngerCategory = False
        AngerAbsRelevance = 0
        AngerRelevance = 0                             
        DisgustCategory = False
        DisgustAbsRelevance = 0
        DisgustRelevance = 0                             
        NoEmotion = True
        
        # Category related process
        #print(result1['category_list'])
        category_emotion_df = pd.json_normalize(result1['category_list'])
        #print(category_emotion_df)
    
        #set the emotion as the index of the category_emotion_pd to make lookup easier                       
        if (category_emotion_df.empty == False): 
            category_emotion_df = category_emotion_df.set_index(category_emotion_df['label'])
            
    
            categories = set(pd.Series(category_emotion_df['label']))
            #print("tones", tones)
    
            #Trust
            if 'Trust' in categories:
                TrustCategory = True
                TrustAbsRelevance = category_emotion_df.loc['Trust', 'abs_relevance']
                TrustRelevance = category_emotion_df.loc['Trust','relevance']
            #Joy            
            if 'Joy' in categories:
                JoyCategory = True
                JoyAbsRelevance = category_emotion_df.loc['Joy', 'abs_relevance']
                JoyRelevance = category_emotion_df.loc['Joy','relevance']

           #Anticipation            
            if 'Anticipation' in categories:
                AnticipationCategory = True
                AnticipationAbsRelevance = category_emotion_df.loc['Anticipation', 'abs_relevance']
                AnticipationRelevance = category_emotion_df.loc['Anticipation','relevance']
                          
            #Surprise            
            if 'Surprise' in categories:
                SurpriseCategory = True
                SurpriseAbsRelevance = category_emotion_df.loc['Surprise', 'abs_relevance']
                SurpriseRelevance = category_emotion_df.loc['Surprise','relevance']
            #Sadness           
            if 'Sadness' in categories:
                SadnessCategory = True
                SadnessAbsRelevance = category_emotion_df.loc['Sadness', 'abs_relevance']
                SadnessRelevance = category_emotion_df.loc['Sadness','relevance']
   
            #Fear            
            if 'Fear' in categories:
                FearCategory = True
                FearAbsRelevance = category_emotion_df.loc['Fear', 'abs_relevance']
                FearRelevance = category_emotion_df.loc['Fear','relevance']
            #Anger      
            if 'Anger' in categories:
                AngerCategory = True
                AngerAbsRelevance = category_emotion_df.loc['Anger', 'abs_relevance']
                AngerRelevance = category_emotion_df.loc['Anger','relevance']
            
            #Disgust            
            if 'Disgust' in categories:
                DisgustCategory = True
                DisgustAbsRelevance = category_emotion_df.loc['Disgust', 'abs_relevance']
                DisgustRelevance = category_emotion_df.loc['Disgust','relevance']
   
            NoEmotion = not (('Trust'in categories) or 
                             ('Joy' in categories) or
                             ('Anticipation' in categories) or
                             ('Surprise' in categories) or
                             ('Sadness' in categories) or
                             ('Fear' in categories) or
                             ('Anger' in categories) or
                             ('Disgust' in categories))
    
    
        category_df = category_df.append({'QuestionID' : row['ID'],
                        'Question Text' : row['Question Text'],
                        'TrustCategory' : TrustCategory,
                        'TrustAbsRelevance': TrustAbsRelevance,
                        'TrustRelevance': TrustRelevance,
                        'JoyCategory':JoyCategory,
                        'JoyAbsRelevance': JoyAbsRelevance,
                        'JoyRelevance':JoyRelevance,
                        'AnticipationCategory': AnticipationCategory,
                        'AnticipationAbsRelevance': AnticipationAbsRelevance, 
                        'AnticipationRelevance': AnticipationRelevance,
                        'SurpriseCategory':SurpriseCategory, 
                        'SurpriseAbsRelevance':SurpriseAbsRelevance,
                        'SurpriseRelevance':SurpriseRelevance,
                        'SadnessCategory':SadnessCategory, 
                        'SadnessAbsRelevance':SadnessAbsRelevance, 
                        'SadnessRelevance':SadnessRelevance,
                        'FearCategory':FearCategory, 
                        'FearAbsRelevance':FearAbsRelevance, 
                        'FearRelevance':FearRelevance,
                        'AngerCategory':AngerCategory, 
                        'AngerAbsRelevance':AngerAbsRelevance, 
                        'AngerRelevance': AngerRelevance,
                        'DisgustCategory':DisgustCategory, 
                        'DisgustAbsRelevance':DisgustAbsRelevance,
                        'DisgustRelevance':DisgustRelevance,
                        'NoEmotion' : NoEmotion}, ignore_index = True)

    
        # term-list related process
    
    
        #get all the term in the term-lists associated with the categories and build the 'term-list' dataframe
        result_pd = pd.DataFrame()
        extract_pd = pd.DataFrame()
        questionid = row['ID']
        questionText = row['Question Text']

        for c_index, c_row in category_emotion_df.iterrows():
            category = c_row.loc['label']
            c_term_list = pd.json_normalize(c_row.loc['term_list'])
            for t_index, t_row in c_term_list.iterrows():
                #print(t_row)
                extract_pd = pd.json_normalize(t_row.loc['offset_list'])
                extract_pd['QuestionID'] = questionid
                extract_pd['Question Text'] = questionText
                extract_pd['Form'] = t_row.loc ['form']
                extract_pd['AbsoluteRelevance'] = t_row.loc['abs_relevance']
                extract_pd['Emotion Label'] = c_row.loc['label']
                termlist_df = termlist_df.append(extract_pd)
                      
 
        if totalCount > 1 and ((totalCount % 5) == 0):
                print("We have processed questions {:,}".format(totalCount))
                category_df.to_csv(categoryfn)
                termlist_df.to_csv(termsfn)
                time_took = time.time() - start_time
                print(f"Total pre-processing runtime: {hms_string(time_took)}")

category_df.to_csv(categoryfn)
termlist_df.to_csv(termsfn)
time_took = time.time() - start_time
print(f"Total pre-processing runtime: {hms_string(time_took)}")



"I want to build a cloud based solution in which I would give a pool of images; and then ask for "find similar image to a particular image from this pool of images"  !! Pool of images can be like "all t-shirt" images. Hence, similar images mean "t-shirt with similar design/color/sleeves" etc.Tagging solution won't work as they are at very high level.AWS Rekognition gives "facial similarities" .. but not "product similarities" .. it does not work like for images of dresses..I am open to use any cloud providers; but all are providing "tags" of the image which won't help me.One solution could be that I use some ML framework like MXNet/Tensorflow, create my own models, train them and then use.. But is there any other ready made solution on any of cloud providers ?"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '2', 'remaining_credits': '19945'}, 'category_list': [{'code': 'Anticipation', 'label': 'Anticipation', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'r

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19938'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I'm pretty new to the google vision api and I am trying to make arequest.Currently I am reading from an image file, encoding it toand trying to pass it on to the requestBut I am getting aerror. Could anyone spot the error in the code?."
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19937'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I am working on a lambda function that needs to access,andservices from AWS.I gaveandpermissions via theand thepolicies respectively and it worked fineThe thing is that I could not access myinstance insidebecause it's inside a VPCI changed my lambda network configurations so it would be able to access the VPC, and theconnection worked as expected, but then the connection tostopped working, whenever I invokefor example it just hangs.Am I missing some per

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19931'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
We have processed questions 15
Total pre-processing runtime: 0:00:23.22
"We are using Google Cloud Vision APIs to extract Invoice fields. We would like to know whether the APIs support detection of table of data? Or do we have to write custom code to detect tables?"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19930'}, 'category_list': [{'code': 'Joy', 'label': 'Joy', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'would like', 'abs_relevance': '1', 'offset_list': [{'inip': '69', 'endp': '78'}]}]}, {'code': 'Trust', 'label': 'Trust', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'support', 'abs_relevance': '1', 'offset_list': [{'inip': '105', 'endp': '111'}]}]}]}
    code  label abs_relevance relevance  \
0    Joy    Joy             1       100   
1  Trust  Trus

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19921'}, 'category_list': [{'code': 'Joy', 'label': 'Joy', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': ':)', 'abs_relevance': '1', 'offset_list': [{'inip': '521', 'endp': '522'}]}]}]}
  code label abs_relevance relevance  \
0  Joy   Joy             1       100   

                                           term_list  
0  [{'form': ':)', 'abs_relevance': '1', 'offset_...  
form                                           :)
abs_relevance                                   1
offset_list      [{'inip': '521', 'endp': '522'}]
Name: 0, dtype: object
"I followed the AWS Rekognition Developer Guide and wrote a stream processor using CreateStreamProcessor in Java.}But I can't figure out how to start the stream processor? Do I have to simply write the main method and callfunction? Or do I have to do something else: like the guide mentioned something as?"
result  {'status': {'code': '0', 'msg': 

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19914'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I'm trying to develop an application that extracts text from a screenshot and with these data (numbers and texts) I do something. It works but not as I expected, it isn't accurate at all. The strange thing is that the same screenshot at the same resolution is recognized in a different way by my application and the "try api" onI noticed that google Keep OCR work better than my app, it use the same api? what can i do for improve the text recognition in my app as google Keep or google vison api site?here is my code:"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '19913'}, 'category_list': [{'code': 'Anticipation', 'label': 'Anticipation', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'expected', 'abs_relevance': '1', 'offset_list': [{'inip': '153', 'endp': '160'}]}]}, {'code': 'Disgu

...

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18491'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I am trying to convert the response from Google Cloud Vision API Client Library to a json format. However i get the following error:ResourceGoogleVision.pythe labels variable is of typeAs you can see i am using message to json function on the labels response. But i am getting the above error.Is there a way to convert the result to a json format?"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18490'}, 'category_list': [{'code': 'Trust', 'label': 'Trust', 'abs_relevance': '2', 'relevance': '100', 'term_list': [{'form': 'labels', 'abs_relevance': '2', 'offset_list': [{'inip': '160', 'endp': '165'}, {'inip': '244', 'endp': '249'}]}]}, {'code': 'Anticipation', 'label': 'Anticipation', 'abs_relevance': '1', 'relevance': '50', 'term_list': [{'form': 'result', 'abs_relevance': '1', 'offset_list': [{'inip':

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18483'}, 'category_list': [{'code': 'Sadness', 'label': 'Sadness', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'crashes', 'abs_relevance': '1', 'offset_list': [{'inip': '284', 'endp': '290'}]}]}]}
      code    label abs_relevance relevance  \
0  Sadness  Sadness             1       100   

                                           term_list  
0  [{'form': 'crashes', 'abs_relevance': '1', 'of...  
form                                      crashes
abs_relevance                                   1
offset_list      [{'inip': '284', 'endp': '290'}]
Name: 0, dtype: object
"I'm working on a project in which I need to analyze an image using Google's Vision API and post the response to a Dynamodb table.I have successfully implemented the Vision API, but not able to convert its response into Python Dictionary.Here's what I have tried:Here's the Vision api implementation:And Here's theFuncti

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18477'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I need to return the message sent by Rekognition to SNS but I get this error in  CloudWatch:Code:And is this the correct way of implementing Rekognition stored video in AWS Lambda with python I didn't find any examples on it.Update:The steps my app needs to take are:In the frontend, the user triggers a lambda function with API gateway which sends a file to s3When the file arrives trigger the same lambda function to apply video recognition and send jobId to SNSwhen SNS receives a message trigger the same lambda function to get the label data and return the data back to the user with API gateway"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18476'}, 'category_list': [{'code': 'Trust', 'label': 'Trust', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'label', 'abs_relevance': '1', 'o

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '2', 'remaining_credits': '18469'}, 'category_list': [{'code': 'Anticipation', 'label': 'Anticipation', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'I wonder', 'abs_relevance': '1', 'offset_list': [{'inip': '1026', 'endp': '1033'}]}]}, {'code': 'Disgust', 'label': 'Disgust', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'did not work', 'abs_relevance': '1', 'offset_list': [{'inip': '359', 'endp': '370'}]}]}, {'code': 'Joy', 'label': 'Joy', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'successful', 'abs_relevance': '1', 'offset_list': [{'inip': '556', 'endp': '565'}]}]}, {'code': 'Trust', 'label': 'Trust', 'abs_relevance': '1', 'relevance': '100', 'term_list': [{'form': 'the best', 'abs_relevance': '1', 'offset_list': [{'inip': '1045', 'endp': '1052'}]}]}]}
           code         label abs_relevance relevance  \
0  Anticipation  Anticipation             1       100   
1 

result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18462'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"I'm trying to read in an image of morse code (dots and dashes) using Google Vision OCR but it's not picking up the symbols very well. Is there a way to make Google Vision pick up the dots and dashes better or is there a different OCR product that can do a better job of recognizing symbols?Currently I am using the nodejs example as provided by Google Cloud at the moment:"
result  {'status': {'code': '0', 'msg': 'OK', 'credits': '1', 'remaining_credits': '18461'}, 'category_list': []}
Empty DataFrame
Columns: []
Index: []
"we are using google vision ocr for gathering text from receipts.In some cases the receipt have some text written in vertical  , like vat information and some other.The question is that  google vision read efficiently only the text in the main orientation (horizontal by example) and discards all the text written in the 